In [ ]:
# load init moudle file

load "/work/pannote/panbot_note.rb"
nil

In [ ]:
puts "==status count=="
puts Task.group(:status).count
puts "==running task=="

puts Task.where(status:"run").map {|x| [x.id,x.tid,x.params,x.name,x.runner,Time.now-x.run_timestamp].join(" | ")}.join("\n")


==status count==
{"abort"=>12, "close"=>2609, "delete"=>2225, "edit"=>37, "open"=>11, "run"=>1}
==running task==
15309 | 4995cb6912fd9a5f | {"trigger_second":"7","min_amount":"0","min_payout":"2.2","bet_amount_factor":"0","bet_amount_value":"0.001"} | panbot_online_runner | panworker-1_6947 | 115857.724227413


In [ ]:
puts Task.where(name:"panbot_simulation_payout").where(status:"close").order(id: :desc).limit(100).map {|x| [x.id,x.name,x.runner,x.updated_at-x.run_timestamp].join(" | ")}.join("\n")
time = Task.where(name:"panbot_simulation_payout").where(status:"close").order(id: :desc).limit(100).map {|x| x.updated_at-x.run_timestamp }
puts time.sum / time.count.to_f


15308 | panbot_simulation_payout | panworker-1_d8c5 | 1.293386
15307 | panbot_simulation_payout |  | 2.070073
15306 | panbot_simulation_payout | panworker-1_c31e | 1.324581
15305 | panbot_simulation_payout | panworker-1_82dd | 1.703764
15304 | panbot_simulation_payout | panworker-1_60b0 | 1.514335
15303 | panbot_simulation_payout | panworker-1_2b03 | 1.668443
15302 | panbot_simulation_payout | panworker-1_d8c5 | 1.319521
15301 | panbot_simulation_payout | panworker-1_c31e | 1.360801
15300 | panbot_simulation_payout |  | 2.023841
15299 | panbot_simulation_payout | panworker-1_82dd | 1.156045
15298 | panbot_simulation_payout | panworker-1_60b0 | 1.283112
15297 | panbot_simulation_payout | panworker-1_2b03 | 1.496839
15296 | panbot_simulation_payout | panworker-1_d8c5 | 1.430735
15295 | panbot_simulation_payout | panworker-1_c31e | 1.288903
15294 | panbot_simulation_payout | panworker-1_82dd | 1.229934
15293 | panbot_simulation_payout |  | 2.105895
15292 | panbot_simulation_payout | panwo

In [ ]:
puts task = Task.where("tid is null").where("status <> ?","delete")
task.each {|x| x.status="delete"; x.save}

KernelInterrupted: Execution interrupted by the Jupyter kernel.

In [ ]:
def abort_reset_to_open()
  Task.where(status:"abort").map {|x| x.status="open"; x.save}
end

def task_clean()
  Task.destroy_all
end

def task_hello_task(count)
  count.times {|x|
    code = <<~TASKCODE

    puts "hello world!"
    _log "hello world!\n"
    
    TASKCODE
    Task.create_task("hello world task-#{x}", code)
  }
end


:task_hello_task

In [ ]:
abort_reset_to_open()

[true]

In [ ]:
puts Task.where(status:"abort").order(:id).map {|x| [x.runner] }

panworker-1_1f56
panworker-1_3307
panworker-1_e289
panworker-1_3307
panworker-1_b418
panworker-1_6947
panworker-1_1f56
panworker-1_38e1
panworker-2_db8b
panworker-1_3307
panworker-1_b418
panworker-1_1f56


In [ ]:
puts Task.where(status:"abort").order(:id).map {|x| [x.id, x.tid ,x.name,x.runner,x.params,x.output] }

11115
54ba06bda99e1105
foo
panworker-1_1f56
{"min_amount":"30","min_payout":"2.8","bet_amount_factor":"0","bet_amount_value":"0.1","epoch_begin":"32630","epoch_end":"32930"}
2022-02-28 17:55:21 +0000 runner panworker-1_1f56 take task 11115 : foo
2022-02-28 17:55:21 +0000 == begin run ==
Exception Class: ArgumentError
Exception Message: wrong number of arguments (given 2, expected 1)
Exception Backtrace:
/panworker/task.rb:42:in `load'
runner_task_closure.rb:10:in `<top (required)>'
/panworker/task.rb:199:in `load'
/panworker/task.rb:199:in `_run'
/panworker/task.rb:208:in `run'
/panworker/task.rb:34:in `block in run_loop'
/panworker/task.rb:31:in `loop'
/panworker/task.rb:31:in `run_loop'
panworker.rb:31:in `<main>'
2022-02-28 17:55:21 +0000 == abort run ==
15539
5c8e5bc0b961f953
demo_error_show_line
panworker-1_3307
{}
2022-02-28 17:52:11 +0000 runner panworker-1_3307 take task 15539 : demo_error_show_line
2022-02-28 17:52:11 +0000 == begin run ==
Exception Class: ZeroDivisionError
Ex

In [ ]:
puts Task.where(status:"close").map {|x| x.output}

2022-02-16 11:14:24 +0000 runner panworker-main2-13 take task 2892 : hello world task-3
2022-02-16 11:14:24 +0000 == begin run ==
hello world!
2022-02-16 11:14:24 +0000 == end run ==
2022-02-16 11:14:21 +0000 runner panworker-main2-13 take task 2889 : hello world task-0
2022-02-16 11:14:21 +0000 == begin run ==
hello world!
2022-02-16 11:14:21 +0000 == end run ==
2022-02-16 11:14:26 +0000 runner panworker-main2-28 take task 2893 : hello world task-4
2022-02-16 11:14:26 +0000 == begin run ==
hello world!
2022-02-16 11:14:26 +0000 == end run ==
2022-02-16 11:14:34 +0000 runner panworker-main2-13 take task 2896 : hello world task-7
2022-02-16 11:14:34 +0000 == begin run ==
hello world!
2022-02-16 11:14:34 +0000 == end run ==
2022-02-16 11:14:40 +0000 runner panworker-main2-28 take task 2898 : hello world task-9
2022-02-16 11:14:40 +0000 == begin run ==
hello world!
2022-02-16 11:14:40 +0000 == end run ==
2022-02-16 11:14:21 +0000 runner panworker-main2-28 take task 2890 : hello world task

In [ ]:
puts Task.where(status:"abort")

In [ ]:
task_hello_task(10)

10

In [ ]:
def task_ping(count)
  count.times {|x|
    code = <<~TASKCODE

    puts "ping"
    
    TASKCODE
    Task.create_task("ping task-#{x}", code)
  }
end

Task.where("name like ?","ping task%").destroy_all
task_ping(50)

50

In [ ]:
puts Task.all.map {|x| [x.status,x.name,x.runner]}.join("\n")

close
hello world task-0
panworker-main2-28
close
hello world task-6
panworker-main2-13
close
hello world task-9
panworker-main2-28
close
hello world task-8
panworker-main2-13
close
hello world task-1
panworker-main2-28
close
hello world task-3
panworker-main2-13
close
hello world task-5
panworker-main2-28
close
hello world task-7
panworker-main2-28
close
hello world task-2
panworker-main2-28
close
hello world task-4
panworker-main2-28


In [ ]:
Task.where("name like ?","ping task%").where(status:"close").where("created_at >= ?",Time.now()-300).select(:runner).distinct(:runner).map {|x| x.runner}

["panworker-main2-13", "panworker-main2-28"]

In [ ]:
code = <<~TASKCODE
    require 'method_source'

    module LocalRunner
        def _log(str)
            print str
        end
    end

    class JobBase
        include LocalRunner
        def self.main_source
            self.instance_method(:main).source
        end
    end

    class MyHelloJob < JobBase
        def main()
            _log ("hello world\n")
            return "value"
        end
    end
TASKCODE

filename = "job.rb"
file = open(filename,"w")
file.write(code)
file.close
load (filename)


job = MyHelloJob.new()
job.main()

Task.create_task("MyHelloJob", MyHelloJob.main_source)


hello world


true

In [ ]:
Task.all

#<ActiveRecord::Relation [#<Task id: 7478, name: "MyHelloJob", status: "close", code: "    def main()\n        _log (\"hello world\n\")\n     ...", output: "2022-02-20 00:35:30 +0000 runner main take task 74...", runner: "main", created_at: "2022-02-20 00:29:29.207230000 +0000", updated_at: "2022-02-20 00:35:30.651661000 +0000", run_timestamp: "2022-02-20 00:35:30.607517000 +0000", return: "value">]>

In [ ]:
Task.destroy_all

[#<Task id: 7476, name: "MyHelloJob", status: "open", code: "    def main()\n        self._log (\"hello world\n\")\n...", output: nil, runner: nil, created_at: "2022-02-20 00:28:51.419154000 +0000", updated_at: "2022-02-20 00:28:51.419154000 +0000", run_timestamp: nil, return: nil>, #<Task id: 7477, name: "MyHelloJob", status: "open", code: "    def main()\n        self._log (\"hello world\n\")\n...", output: nil, runner: nil, created_at: "2022-02-20 00:29:15.310541000 +0000", updated_at: "2022-02-20 00:29:15.310541000 +0000", run_timestamp: nil, return: nil>]

In [ ]:
eval(MyHelloJob.main_source + "main()")

NoMethodError: undefined method `_log' for #<Object:0x000055856cf1d258>

In [ ]:
MyHelloJob.main_source + "main()"

"    def main()\n        _log (\"hello world\n\")\n        return \"value\"\n    end\nmain()"

In [ ]:
def a
 return 3
end

:a

In [ ]:
puts Task.last.output

2022-02-22 14:12:40 +0000 runner panworker-1_72df take task 9908 : ping task-49
2022-02-22 14:12:40 +0000 == begin run ==
ping
Exception Class: NoMethodError
Exception Message: undefined method `main' for #<Task::Runner:0x000055d42eced300>
Exception Backtrace:
(eval):13:in `_run'
(eval):9:in `eval'
(eval):9:in `_run'
/panworker/task.rb:105:in `run'
/panworker/task.rb:28:in `block in run_loop'
/panworker/task.rb:26:in `loop'
/panworker/task.rb:26:in `run_loop'
panworker.rb:31:in `<main>'
2022-02-22 14:12:40 +0000 == abort run ==


In [ ]:
puts Task.last.code

_log "ping
"


In [ ]:
            code = <<~'TASKCODE'
            def main
                _log "ping\n"
            end
            TASKCODE


"def main\n    _log \"ping\\n\"\nend\n"

In [ ]:
puts code

def main
    _log "ping\n"
end


In [ ]:
Task.where("tid is not null").where("name not like ?","%demo%").destroy_all

[#<Task id: 9741, name: "hello-task", status: "edit", code: "__TASK_NAME__ = \"hello-task\"\n\ndef main()\n    time ...", output: nil, runner: nil, created_at: "2022-02-22 03:50:46.278510000 +0000", updated_at: "2022-02-22 03:50:46.278510000 +0000", return: nil, tid: "87c4c144bb74c524", params: "{}", save_timestamp: nil, run_timestamp: nil>, #<Task id: 9742, name: "hello-task", status: "edit", code: "__TASK_NAME__ = \"hello-task\"\n\ndef main()\n    time ...", output: nil, runner: nil, created_at: "2022-02-22 03:50:50.560716000 +0000", updated_at: "2022-02-22 03:50:50.560716000 +0000", return: nil, tid: "9588269cc457df46", params: "{}", save_timestamp: nil, run_timestamp: nil>, #<Task id: 9753, name: "hello-task", status: "edit", code: "__TASK_NAME__ = \"hello-task\"\n\ndef main()\n    time ...", output: nil, runner: nil, created_at: "2022-02-22 08:42:29.266752000 +0000", updated_at: "2022-02-22 08:42:29.266752000 +0000", return: nil, tid: "b98d4645652a7afb", params: "{}", save_timesta

In [ ]:
puts Task.where(status:"close").where(name:"panbot_simluation_payout").map { |x| x.params}.join("\n")

Interrupt: 

In [ ]:
Task.where(status:"abort").map {|x| x.status="delete"; x.save}

[true, true]

In [ ]:
Cache.where("key like ?","%{%").map {|x| x.delete}

[]

In [ ]:
Log.destroy_all

[#<Log id: 2117, pid: 1, log: "epoch 48108 | tick 82 | now 2022-02-25 14:36:52 | ...", created_at: "2022-02-25 14:36:52.421460000 +0000", updated_at: "2022-02-25 14:36:52.421460000 +0000", worker: "panworker-0_c17d", hostname: "4994ef3ca4af", ip_addr: "172.17.0.6">, #<Log id: 2118, pid: 1, log: "epoch 48108 | tick 83 | now 2022-02-25 14:36:52 | ...", created_at: "2022-02-25 14:36:52.444802000 +0000", updated_at: "2022-02-25 14:36:52.444802000 +0000", worker: "panworker-0_c17d", hostname: "4994ef3ca4af", ip_addr: "172.17.0.6">, #<Log id: 2119, pid: 1, log: "epoch 48108 | tick 84 | now 2022-02-25 14:36:52 | ...", created_at: "2022-02-25 14:36:52.543654000 +0000", updated_at: "2022-02-25 14:36:52.543654000 +0000", worker: "panworker-0_c17d", hostname: "4994ef3ca4af", ip_addr: "172.17.0.6">, #<Log id: 3409, pid: 1, log: "do not bet\n", created_at: "2022-02-25 15:43:47.707232000 +0000", updated_at: "2022-02-25 15:43:47.707232000 +0000", worker: "panworker-0_6781", hostname: "7ad09d8756a1", 

In [ ]:
Log.where("worker <> ?","panworker-0_c17d").map {|x| x.delete}

[#<Log id: 4603, pid: 1, log: "epoch 48133 | tick 0 | now 2022-02-25 16:40:50 | l...", created_at: "2022-02-25 16:40:50.115256000 +0000", updated_at: "2022-02-25 16:40:50.115256000 +0000", worker: "panworker-0_8ce3", hostname: "42cb1d281e84", ip_addr: "172.17.0.5">, #<Log id: 4604, pid: 1, log: "epoch 48133 | tick 1 | now 2022-02-25 16:41:20 | l...", created_at: "2022-02-25 16:41:20.150404000 +0000", updated_at: "2022-02-25 16:41:20.150404000 +0000", worker: "panworker-0_8ce3", hostname: "42cb1d281e84", ip_addr: "172.17.0.5">, #<Log id: 4605, pid: 1, log: "epoch 48133 | tick 2 | now 2022-02-25 16:41:50 | l...", created_at: "2022-02-25 16:41:50.170787000 +0000", updated_at: "2022-02-25 16:41:50.170787000 +0000", worker: "panworker-0_8ce3", hostname: "42cb1d281e84", ip_addr: "172.17.0.5">, #<Log id: 4606, pid: 1, log: "epoch 48133 | tick 3 | now 2022-02-25 16:42:20 | l...", created_at: "2022-02-25 16:42:20.147946000 +0000", updated_at: "2022-02-25 16:42:20.147946000 +0000", worker: "panw

In [ ]:
Log.first.delete

#<Log id: 4728, pid: 1, log: "=== Bot Address: 0x358306f2c32537DAC8c8F74c70D19A0...", created_at: "2022-02-25 16:55:50.925022000 +0000", updated_at: "2022-02-25 16:55:50.925022000 +0000", worker: "panworker-0_c17d", hostname: "4994ef3ca4af", ip_addr: "172.17.0.6">

In [ ]:
Task.where(status:"run").map {|x| x.status="abort"; x.save}

[true, true, true, true, true, true, true, true]

In [ ]:
Block.count

2334367

In [ ]:
Block.count

2334367

In [ ]:
Cache.find_by_key("Block-count").delete

#<Cache id: 2045, key: "Block-count", value_type: "Integer", value_s: nil, value_i: 2334367, value_f: nil, value_t: nil, value_b: nil, created_at: "2022-02-26 15:32:52.305872000 +0000", updated_at: "2022-02-26 15:32:52.305872000 +0000">

In [ ]:
Task.find(15390).delete

#<Task id: 15390, name: "block_data_import", status: "abort", code: "__TASK_NAME__ = \"block_data_import\"\n\n\nrequire 'par...", output: "2022-02-26 16:08:20 +0000 runner panworker-0_100a ...", runner: "panworker-0_100a", created_at: "2022-02-26 14:35:18.980077000 +0000", updated_at: "2022-02-26 16:08:27.143324000 +0000", return: nil, tid: nil, params: "{\"block_begin\":\"15590001\",\"block_end\":\"15596755\"}", save_timestamp: "2022-02-26 14:35:18.979008000 +0000", run_timestamp: "2022-02-26 16:08:20.659358000 +0000">

In [ ]:
Block.where("block_number > ?",15600000).count
Tx.where("block_number > ?",15600000).count
Event.where("block_number > ?",15600000).count
EpochDetail.where("block_number > ?",15600000).count


14783

In [ ]:
Task.find_by_tid("4995cb6912fd9a5f").output.size

1881043

In [ ]:
a="1"*123456

"..."

In [ ]:
a.size

123456

In [ ]:
a[-1024,1024]

"111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [ ]:
a="1234567"

"1234567"

In [ ]:
a[-3,3]

"567"

In [ ]:
# Block.where("block_number > ?",15600000).count
# Tx.where("block_number > ?",15600000).count
# Event.where("block_number > ?",15600000).count
# EpochDetail.where("block_number > ?",15600000).count
# Epoch.where("epoch >= ?",Epoch.where("lock_block_number > ?",15600000).order(:epoch).first.epoch-1).count


0

In [ ]:
t=Task.find(15318)
t.status = "delete"
t.save

true

In [ ]:
Task.where("id=15434").map {|x| x.delete}

[#<Task id: 15434, name: "demo_factorial", status: "run", code: "__TASK_NAME__ = \"demo_factorial\"\n\n\ndef main()\n    ...", output: nil, runner: "panworker-0_2063", created_at: "2022-02-26 14:35:16.124442000 +0000", updated_at: "2022-02-27 08:54:58.323728000 +0000", return: nil, tid: nil, params: "{\"n\":\"1\"}", save_timestamp: "2022-02-27 08:54:57.612059000 +0000", run_timestamp: "2022-02-27 08:54:58.319066000 +0000">]

In [ ]:
Task.where("tid is null").where(name:"panbot/simulation/panbot_simulation_payout").map {|x| x.status='delete'; x.save}

[true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, true, tru

In [ ]:
x= Task.where(status:"abort").all[0]
x.status="open"
x.save

true

In [ ]:
Cache.limit(1).offset(100)

#<ActiveRecord::Relation [#<Cache id: 235, key: "Group-human-stats", value_type: "JSON", value_s: "{\"bet_epoch_cnt\":295,\"invest_cnt\":260,\"bet_cnt\":29...", value_i: nil, value_f: nil, value_t: nil, value_b: nil, created_at: "2022-02-18 14:13:51.357072000 +0000", updated_at: "2022-02-18 14:13:51.357072000 +0000">]>

In [ ]:
Cache.where("key like ?","Task::Runner%").map {|x| x.delete}

[#<Cache id: 742, key: "Task::Runner::PayoutIntervalBot{\"min_amount\":0,\"mi...", value_type: "JSON", value_s: "[{\"epoch\":32630,\"bet\":\"\",\"amount\":null,\"bet_block_...", value_i: nil, value_f: nil, value_t: nil, value_b: nil, created_at: "2022-02-24 12:30:49.784535000 +0000", updated_at: "2022-02-24 12:30:49.784535000 +0000">, #<Cache id: 755, key: "Task::Runner::PayoutIntervalBot{\"min_amount\":1,\"mi...", value_type: "JSON", value_s: "[{\"epoch\":32630,\"bet\":\"\",\"amount\":null,\"bet_block_...", value_i: nil, value_f: nil, value_t: nil, value_b: nil, created_at: "2022-02-24 12:37:56.880118000 +0000", updated_at: "2022-02-24 12:37:56.880118000 +0000">, #<Cache id: 768, key: "Task::Runner::PayoutIntervalBot{\"min_amount\":1,\"mi...", value_type: "JSON", value_s: "[{\"epoch\":32630,\"bet\":\"\",\"amount\":null,\"bet_block_...", value_i: nil, value_f: nil, value_t: nil, value_b: nil, created_at: "2022-02-24 12:38:11.681662000 +0000", updated_at: "2022-02-24 12:38:11.681662000 

In [ ]:
Task.where(status:"kill").last

#<Task id: 16847, name: "data_import/transfer_data_import", status: "kill", code: "__TASK_NAME__= \"data_import/transfer_data_import\"\n...", output: "2022-03-03 23:05:40 +0000 runner panworker-1_62b6 ...", runner: "panworker-1_62b6", created_at: "2022-02-27 07:55:09.226876000 +0000", updated_at: "2022-03-03 23:07:25.690575000 +0000", return: "{\"raw_ret\":null,\"html\":\"\"}", tid: nil, params: "{\"block_begin\":\"15277000\",\"block_end\":\"15278000\"}", save_timestamp: "2022-03-03 23:01:20.749643000 +0000", run_timestamp: "2022-03-03 23:05:40.597995000 +0000", schedule_at: "1970-01-01 00:00:00.000000000 +0000">

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=ebde59c2-d67b-4e31-ad3b-1bdd07014691' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>